# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
document = ET.parse( './data/mondial_database.xml' )
import pandas as pd
import numpy as np

### Exercise 1

In [7]:
cim = pd.DataFrame([ [country.find('name').text, 
              np.nan if country.find('infant_mortality') is None else float(country.find('infant_mortality').text)] 
              for country in document.findall( 'country' )],
            columns = ['Country', 'Infant Mortality'])
cim.sort_values('Infant Mortality')[0:10]

,Country,Infant Mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


### Exercise 2

In [8]:
cp = pd.DataFrame([ [city.find('name').text, 
    np.nan if city.find('population') is None else int(city.find('population').text)] 
    for city in  document.findall( 'country/city' ) + document.findall( 'country/province/city' )],
            columns = ['City','Population'])
cp.sort_values('Population',ascending=False)[0:10]

,City,Population
176,Seoul,10229262.0
1885,Mumbai,9925891.0
2920,São Paulo,9412894.0
2021,Jakarta,8259266.0
1708,Shanghai,8205598.0
2299,Ciudad de México,8092449.0
888,Moskva,8010954.0
2130,Tokyo,7843000.0
1707,Beijing,7362426.0
1940,Delhi,7206704.0


## Exercise 3

In [9]:
# Initialize data frame of ethnic groups
egs = [eg.text for eg in document.findall( 'country/ethnicgroup' )]
egs = list(pd.Series(egs).drop_duplicates())
egs_ser = pd.Series([0]*len(egs),index=egs)

# Sum over all countries
for c in document.findall('country'):
    
    # Get most recent population
    population = 0
    year = 0
    for p in c.findall('population'):
        newyear = int(p.get('year'))
        if newyear > year:
            year = newyear
            population = int(p.text)
            
    # Sweep over all ethnic groups in country
    for e in c.findall('ethnicgroup'):
        egs_ser.loc[e.text] += int(float(e.get('percentage'))*population/100)

# Print top ten
print egs_ser.sort_values(ascending=False)[0:10]


Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
dtype: int64


### Exercise 4

Lakes and rivers can have multiple countries so I just returned the string of country codes. Given we only had to do the top for each, seems too much work to convert codes with Python, when you can just read them off yourself (Colombia, Brazil, Peru for part (a), Russia, Azerbaijan, Kazakhstan, Iran, Turkmenistan for part (b), Bolivia for part (c))

In [18]:
rivers = pd.DataFrame([[r.find('name').text, 
    np.nan if r.find('length') is None else float(r.find('length').text),
    r.get('country')] for r in document.findall('river')],
    columns=['River', 'Length','Country code'])

rivers.sort_values('Length',ascending = False)[0:1]

,River,Length,Country code
174,Amazonas,6448.0,CO BR PE


In [19]:
lakes = pd.DataFrame([[l.find('name').text, 
    np.nan if l.find('area') is None else float(l.find('area').text),
    l.get('country')] for l in document.findall('lake')],
    columns=['Lake', 'Area','Country code'])

lakes.sort_values('Area',ascending = False)[0:1]

,Lake,Area,Country code
54,Caspian Sea,386400.0,R AZ KAZ IR TM


In [20]:
airports = pd.DataFrame([[a.find('name').text, 
      np.nan if a.find('elevation').text is None else int(a.find('elevation').text),
      a.get('country')] for a in document.findall('airport')],
      columns=['Airport', 'Elevation', 'Country code'])

airports.sort_values('Elevation',ascending = False)[0:1]

,Airport,Elevation,Country code
80,El Alto Intl,4063.0,BOL
